这篇博客（[视频](https://www.bilibili.com/video/BV1fA41167Qg/?spm_id_from=333.999.0.0&vd_source=b4496ba547455f3db613a8f8de6bea01)）记录从巨潮资讯爬取区块链技术投资相关的公告数据。
<!-- TEASER_END -->

In [1]:
import logging
import time

import numpy as np
import pandas as pd
import requests

from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_json(pagenum, s):
    """获取 json 数据

    Args:
        pagenum (str): page number
        s (requests.Session): Session object

    Returns:
        res.json(): json data
    """
    url = 'http://www.cninfo.com.cn/new/fulltextSearch/full?'
    
    headers = {
        "Accept-Encoding": "gzip",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Connection": "keep-alive",
    }
    
    payload = {
        'searchkey': '区块链',
        'sdate': '2009-01-01',
        'edate': '2022-01-01',
        'isfulltext': 'true',
        'sortName': 'pubdate',
        'sortType': 'asc',
        'type': 'shj',
    }
    
    payload['pageNum'] = str(pagenum)
    
    try:
        res = s.get(url, headers=headers, params=payload)
    except Exception as e:
        logging.warning(e)
    

    return res.json()

In [3]:
with requests.Session() as s:
    page_one = pd.DataFrame(get_json(1, s)['announcements'])

In [5]:
page_one.head()

,id,secCode,secName,orgId,announcementId,announcementTitle,announcementTime,adjunctUrl,adjunctSize,adjunctType,storageTime,columnId,pageColumn,announcementType,associateAnnouncement,important,batchNum,announcementContent,orgName,announcementTypeName
0,None,000631,顺发恒业,gssz0000631,49624621,*ST 兰宝：重大资产出售、发行股份购买资产暨关联交易报告书（修订稿）,1235773830000,finalpage/2009-02-28/49624621.PDF,1749,PDF,None,01030101||010612,SZZB,01010501||01010701||010112||01170110,None,None,None,（二）公司业绩持续下滑，且无改善迹象 由于个别银行发现公司资金<em>链</em>紧张，逐步...,None,None
1,None,600596,新安股份,gssh0600596,49988949,新安股份：2008年年度报告,1236637800000,finalpage/2009-03-10/49988949.PDF,1711,PDF,None,01030103||01030404||010612||010613,SHZB,01010501||010113||01030101,None,None,None,3.新安包装公司搬迁事项 因新安江桥东<em>区块</em>旧城改造，新安包装公司于本期从老...,None,None
2,None,600267,海正药业,gssh0600267,50367093,海正药业：2008年年度股东大会会议资料,1237501800000,finalpage/2009-03-20/50367093.PDF,346,PDF,None,01030103||010612,SHZB,01010501||010113||011906,None,None,None,"100.00% 141,278 100.00% 115,132 100.00% 公司凭借原料...",None,None
3,None,002244,滨江集团,9900004730,50327309,滨江集团：2008年年度报告,1237501800000,finalpage/2009-03-20/50327309.PDF,867,PDF,None,01010302||01010306||01010410||01010411||010301...,SZZB,01010503||010112||010114||01030101,None,None,None,江干科技经济园区地块开发协议书》（江科园协字【2006】035 号），约定就“S08、09、...,None,None
4,None,000301,东方盛虹,gssz0000301,50432137,东方市场：2008年年度报告,1237847400000,finalpage/2009-03-24/50432137.PDF,369,PDF,None,01030101||01030402||010612||010613,SZZB,01010501||010112||01030101,None,None,None,公司将采取措施进一步完善产业<em>链</em>，继续减少关联交易。,None,None


In [6]:
column = page_one.columns

In [7]:
def get_data_array(pagenum, s):
    announcements_list = get_json(pagenum, s)['announcements']
    return pd.DataFrame(announcements_list).values

In [8]:
pages = range(1, 1106)
res =  []
with ThreadPoolExecutor(max_workers=10) as pool:
    with requests.Session() as s:
        futures = [pool.submit(get_data_array, pagenum, s) for pagenum in pages]
        for future in futures:
            res.extend(future.result())

In [9]:
res_df = pd.DataFrame(res, columns=column)

In [10]:
res_df.head()

,id,secCode,secName,orgId,announcementId,announcementTitle,announcementTime,adjunctUrl,adjunctSize,adjunctType,storageTime,columnId,pageColumn,announcementType,associateAnnouncement,important,batchNum,announcementContent,orgName,announcementTypeName
0,None,000631,顺发恒业,gssz0000631,49624621,*ST 兰宝：重大资产出售、发行股份购买资产暨关联交易报告书（修订稿）,1235773830000,finalpage/2009-02-28/49624621.PDF,1749,PDF,None,01030101||010612,SZZB,01010501||01010701||010112||01170110,None,None,None,（二）公司业绩持续下滑，且无改善迹象 由于个别银行发现公司资金<em>链</em>紧张，逐步...,None,None
1,None,600596,新安股份,gssh0600596,49988949,新安股份：2008年年度报告,1236637800000,finalpage/2009-03-10/49988949.PDF,1711,PDF,None,01030103||01030404||010612||010613,SHZB,01010501||010113||01030101,None,None,None,3.新安包装公司搬迁事项 因新安江桥东<em>区块</em>旧城改造，新安包装公司于本期从老...,None,None
2,None,600267,海正药业,gssh0600267,50367093,海正药业：2008年年度股东大会会议资料,1237501800000,finalpage/2009-03-20/50367093.PDF,346,PDF,None,01030103||010612,SHZB,01010501||010113||011906,None,None,None,"100.00% 141,278 100.00% 115,132 100.00% 公司凭借原料...",None,None
3,None,002244,滨江集团,9900004730,50327309,滨江集团：2008年年度报告,1237501800000,finalpage/2009-03-20/50327309.PDF,867,PDF,None,01010302||01010306||01010410||01010411||010301...,SZZB,01010503||010112||010114||01030101,None,None,None,江干科技经济园区地块开发协议书》（江科园协字【2006】035 号），约定就“S08、09、...,None,None
4,None,000301,东方盛虹,gssz0000301,50432137,东方市场：2008年年度报告,1237847400000,finalpage/2009-03-24/50432137.PDF,369,PDF,None,01030101||01030402||010612||010613,SZZB,01010501||010112||01030101,None,None,None,公司将采取措施进一步完善产业<em>链</em>，继续减少关联交易。,None,None


In [11]:
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11050 entries, 0 to 11049
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     0 non-null      object
 1   secCode                11050 non-null  object
 2   secName                11050 non-null  object
 3   orgId                  11050 non-null  object
 4   announcementId         11050 non-null  object
 5   announcementTitle      11050 non-null  object
 6   announcementTime       11050 non-null  int64 
 7   adjunctUrl             11050 non-null  object
 8   adjunctSize            11050 non-null  int64 
 9   adjunctType            11050 non-null  object
 10  storageTime            0 non-null      object
 11  columnId               11050 non-null  object
 12  pageColumn             11050 non-null  object
 13  announcementType       11050 non-null  object
 14  associateAnnouncement  910 non-null    object
 15  important          

In [12]:
raw_data = res_df.drop(
    columns=['id', 'announcementId', 'adjunctSize', 'adjunctType', 'storageTime', 'columnId', 'pageColumn',
             'announcementType', 'associateAnnouncement', 'important', 'batchNum', 'orgName', 'announcementTypeName'],
)

In [13]:
raw_data

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
0,000631,顺发恒业,gssz0000631,*ST 兰宝：重大资产出售、发行股份购买资产暨关联交易报告书（修订稿）,1235773830000,finalpage/2009-02-28/49624621.PDF,（二）公司业绩持续下滑，且无改善迹象 由于个别银行发现公司资金<em>链</em>紧张，逐步...
1,600596,新安股份,gssh0600596,新安股份：2008年年度报告,1236637800000,finalpage/2009-03-10/49988949.PDF,3.新安包装公司搬迁事项 因新安江桥东<em>区块</em>旧城改造，新安包装公司于本期从老...
2,600267,海正药业,gssh0600267,海正药业：2008年年度股东大会会议资料,1237501800000,finalpage/2009-03-20/50367093.PDF,"100.00% 141,278 100.00% 115,132 100.00% 公司凭借原料..."
3,002244,滨江集团,9900004730,滨江集团：2008年年度报告,1237501800000,finalpage/2009-03-20/50327309.PDF,江干科技经济园区地块开发协议书》（江科园协字【2006】035 号），约定就“S08、09、...
4,000301,东方盛虹,gssz0000301,东方市场：2008年年度报告,1237847400000,finalpage/2009-03-24/50432137.PDF,公司将采取措施进一步完善产业<em>链</em>，继续减少关联交易。
...,...,...,...,...,...,...,...
11045,600382,*ST广珠,gssh0600382,*ST广珠：西部证券股份有限公司关于广东明珠集团股份有限公司重大资产出售及购买资产暨关联交易...,1640707200000,finalpage/2021-12-29/1212047512.PDF,、探矿权、采矿权规定方面，相关法律法规主要有《中华人民共和国矿产资源法》及其实施细则、《矿产...
11046,300164,通源石油,9900016228,通源石油：关于2021年以简易程序向特定对象发行股票募集资金使用可行性分析报告（修订稿）,1640776515000,finalpage/2021-12-29/1212055672.PDF,公司是以油气田增产为核心的油服企业，业务涵盖钻井、定向、完井压裂、测井、射孔、带压作业、连续...
11047,300164,通源石油,9900016228,通源石油：关于2021年以简易程序向特定对象发行股票方案论证分析报告（修订稿）,1640776515000,finalpage/2021-12-29/1212055670.PDF,公司是以油气田增产为核心的国际油服企业，业务涵盖钻井、定向、完井压裂、测井、射孔、带压作业、...
11048,300164,通源石油,9900016228,通源石油：2021年以简易程序向特定对象发行股票预案（修订稿）,1640776516000,finalpage/2021-12-29/1212055680.PDF,公司是以油气田增产为核心的油服企业，业务涵盖钻井、定向、完井压裂、测井、射孔、带压作业、连续...


In [14]:
# 去除 announcementTitle 和 announcementContent 里的 <em> 标签
def drop_em(c): 
    return c.replace('<em>', '').replace('</em>', '')

data_drop_em = raw_data.copy()
data_drop_em.loc[:, 'announcementTitle'] = data_drop_em.loc[:, 'announcementTitle'].apply(lambda c : drop_em(c))
data_drop_em.loc[:, 'announcementContent'] = data_drop_em.loc[:, 'announcementContent'].apply(lambda c : drop_em(c))

In [15]:
data_drop_em.head()

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
0,000631,顺发恒业,gssz0000631,*ST 兰宝：重大资产出售、发行股份购买资产暨关联交易报告书（修订稿）,1235773830000,finalpage/2009-02-28/49624621.PDF,（二）公司业绩持续下滑，且无改善迹象 由于个别银行发现公司资金链紧张，逐步停止转贷并收缩银根...
1,600596,新安股份,gssh0600596,新安股份：2008年年度报告,1236637800000,finalpage/2009-03-10/49988949.PDF,3.新安包装公司搬迁事项 因新安江桥东区块旧城改造，新安包装公司于本期从老厂区（新安江街道桥...
2,600267,海正药业,gssh0600267,海正药业：2008年年度股东大会会议资料,1237501800000,finalpage/2009-03-20/50367093.PDF,"100.00% 141,278 100.00% 115,132 100.00% 公司凭借原料..."
3,002244,滨江集团,9900004730,滨江集团：2008年年度报告,1237501800000,finalpage/2009-03-20/50327309.PDF,江干科技经济园区地块开发协议书》（江科园协字【2006】035 号），约定就“S08、09、...
4,000301,东方盛虹,gssz0000301,东方市场：2008年年度报告,1237847400000,finalpage/2009-03-24/50432137.PDF,公司将采取措施进一步完善产业链，继续减少关联交易。


In [16]:
# 将时间戳转化为日期
def transform_date(timeStamp):
    timeStamp = int(str(timeStamp)[:-3])
    timeArray = time.localtime(timeStamp)
    return time.strftime("%Y-%m-%d", timeArray)

data_time_transformed = data_drop_em.copy()
data_time_transformed.loc[:, 'announcementTime'] = data_time_transformed.loc[:, 'announcementTime'].apply(lambda c: transform_date(c))
data_time_transformed.head()

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
0,000631,顺发恒业,gssz0000631,*ST 兰宝：重大资产出售、发行股份购买资产暨关联交易报告书（修订稿）,2009-02-28,finalpage/2009-02-28/49624621.PDF,（二）公司业绩持续下滑，且无改善迹象 由于个别银行发现公司资金链紧张，逐步停止转贷并收缩银根...
1,600596,新安股份,gssh0600596,新安股份：2008年年度报告,2009-03-10,finalpage/2009-03-10/49988949.PDF,3.新安包装公司搬迁事项 因新安江桥东区块旧城改造，新安包装公司于本期从老厂区（新安江街道桥...
2,600267,海正药业,gssh0600267,海正药业：2008年年度股东大会会议资料,2009-03-20,finalpage/2009-03-20/50367093.PDF,"100.00% 141,278 100.00% 115,132 100.00% 公司凭借原料..."
3,002244,滨江集团,9900004730,滨江集团：2008年年度报告,2009-03-20,finalpage/2009-03-20/50327309.PDF,江干科技经济园区地块开发协议书》（江科园协字【2006】035 号），约定就“S08、09、...
4,000301,东方盛虹,gssz0000301,东方市场：2008年年度报告,2009-03-24,finalpage/2009-03-24/50432137.PDF,公司将采取措施进一步完善产业链，继续减少关联交易。


In [17]:
# 拼接公告 url
def url_join(c):
    return ''.join(['http://static.cninfo.com.cn/', c])

data_url_joined = data_time_transformed.copy()
data_url_joined.loc[:, 'adjunctUrl'] = data_url_joined.loc[:, 'adjunctUrl'].apply(lambda c: url_join(c))
data_url_joined.head()

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
0,000631,顺发恒业,gssz0000631,*ST 兰宝：重大资产出售、发行股份购买资产暨关联交易报告书（修订稿）,2009-02-28,http://static.cninfo.com.cn/finalpage/2009-02-...,（二）公司业绩持续下滑，且无改善迹象 由于个别银行发现公司资金链紧张，逐步停止转贷并收缩银根...
1,600596,新安股份,gssh0600596,新安股份：2008年年度报告,2009-03-10,http://static.cninfo.com.cn/finalpage/2009-03-...,3.新安包装公司搬迁事项 因新安江桥东区块旧城改造，新安包装公司于本期从老厂区（新安江街道桥...
2,600267,海正药业,gssh0600267,海正药业：2008年年度股东大会会议资料,2009-03-20,http://static.cninfo.com.cn/finalpage/2009-03-...,"100.00% 141,278 100.00% 115,132 100.00% 公司凭借原料..."
3,002244,滨江集团,9900004730,滨江集团：2008年年度报告,2009-03-20,http://static.cninfo.com.cn/finalpage/2009-03-...,江干科技经济园区地块开发协议书》（江科园协字【2006】035 号），约定就“S08、09、...
4,000301,东方盛虹,gssz0000301,东方市场：2008年年度报告,2009-03-24,http://static.cninfo.com.cn/finalpage/2009-03-...,公司将采取措施进一步完善产业链，继续减少关联交易。


In [18]:
data_url_joined.to_csv('./data/raw_data.csv', index=False, encoding='utf_8_sig')